**Write a program to generate confusion matrices for different models and analyze their
performance.**

Step 1: Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


Step 2: Load and Prepare Iris Dataset

In [ ]:
# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Step 3: Implement Logistic Regression Model

In [ ]:
class LogisticRegression:
    def __init__(self, learning_rate=0.01, epochs=1000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def fit(self, X, y):
        num_samples, num_features = X.shape
        self.weights = np.zeros(num_features)
        self.bias = 0

        for _ in range(self.epochs):
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self.sigmoid(linear_model)

            # Compute gradients
            dw = (1 / num_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / num_samples) * np.sum(y_predicted - y)

            # Update parameters
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self.sigmoid(linear_model)
        y_predicted_cls = np.where(y_predicted > 0.5, 1, 0)

        # Since Iris is multi-class, we need to adjust the prediction logic
        # Here, we'll use a simple approach for demonstration purposes
        # In practice, you would use a one-vs-all or one-vs-one strategy
        y_predicted_cls = np.argmax(np.array([y_predicted, 1 - y_predicted, np.zeros_like(y_predicted)]).T, axis=1)

        return y_predicted_cls

    def evaluate(self, X, y):
        predictions = self.predict(X)
        accuracy = np.mean(predictions == y)
        return accuracy


Step 4: Implement Neural Network Model

In [ ]:
class NeuralNetwork:
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        # Initialize weights randomly
        self.weights1 = np.random.rand(input_dim, hidden_dim)
        self.bias1 = np.zeros((1, hidden_dim))
        self.weights2 = np.random.rand(hidden_dim, output_dim)
        self.bias2 = np.zeros((1, output_dim))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def relu(self, x):
        return np.maximum(x, 0)

    def softmax(self, x):
        exp_x = np.exp(x)
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def forward(self, X):
        self.hidden_layer = np.dot(X, self.weights1) + self.bias1
        self.hidden_layer = self.relu(self.hidden_layer)

        self.output_layer = np.dot(self.hidden_layer, self.weights2) + self.bias2
        self.output_layer = self.softmax(self.output_layer)

        return self.output_layer

    def cross_entropy_loss(self, predictions, labels):
        return -np.mean(np.sum(labels * np.log(predictions), axis=1))

    def train(self, X, y, learning_rate=0.01, epochs=1000):
        for epoch in range(epochs):
            predictions = self.forward(X)
            loss = self.cross_entropy_loss(predictions, y)

            # Backpropagation
            d_output = predictions - y
            d_weights2 = np.dot(self.hidden_layer.T, d_output)
            d_bias2 = np.sum(d_output, axis=0, keepdims=True)

            d_hidden_layer = d_output.dot(self.weights2.T) * (self.hidden_layer > 0).astype(int)
            d_weights1 = np.dot(X.T, d_hidden_layer)
            d_bias1 = np.sum(d_hidden_layer, axis=0, keepdims=True)

            # Update weights and biases
            self.weights1 -= learning_rate * d_weights1
            self.bias1 -= learning_rate * d_bias1
            self.weights2 -= learning_rate * d_weights2
            self.bias2 -= learning_rate * d_bias2

            if epoch % 100 == 0:
                print(f'Epoch {epoch+1}, Loss: {loss}')

    def predict(self, X):
        predictions = self.forward(X)
        return np.argmax(predictions, axis=1)

    def evaluate(self, X, y):
        predictions = self.predict(X)
        accuracy = np.mean(predictions == np.argmax(y, axis=1))
        return accuracy


Step 5: Generate Confusion Matrices

In [ ]:
from sklearn.metrics import confusion_matrix

# Train logistic regression model
# Note: Logistic regression needs adjustment for multi-class classification
# Here, we focus on the neural network for multi-class

# Train neural network model
nn_model = NeuralNetwork(4, 10, 3)
y_train_onehot = np.eye(3)[y_train]
nn_model.train(X_train, y_train_onehot)

# Predict using neural network
y_pred_nn = nn_model.predict(X_test)

# Generate confusion matrix for neural network
conf_mat_nn = confusion_matrix(y_test, y_pred_nn)
print("Confusion Matrix (Neural Network):\n", conf_mat_nn)

# For logistic regression, we need to adjust it for multi-class
# Here, we skip it due to its complexity in this context

# However, if you want to use logistic regression for multi-class,
# consider using one-vs-all or one-vs-one strategies.

# To simplify, let's use a basic form of logistic regression
# and acknowledge its limitations for multi-class problems.

# Train logistic regression (simplified for demonstration)
lr_model = LogisticRegression()
# Note: This won't work well for multi-class without adjustments

# For multi-class, consider using sklearn's LogisticRegression
from sklearn.linear_model import LogisticRegression as SKLogisticRegression
sk_lr_model = SKLogisticRegression(max_iter=1000)
sk_lr_model.fit(X_train, y_train)

# Predict using sklearn's logistic regression
y_pred_sk_lr = sk_lr_model.predict(X_test)

# Generate confusion matrix for sklearn's logistic regression
conf_mat_sk_lr = confusion_matrix(y_test, y_pred_sk_lr)
print("Confusion Matrix (sklearn Logistic Regression):\n", conf_mat_sk_lr)
